# **DeepGA Modificado**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Información del sistema de ejecución

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

/bin/bash: nvidia-smi: command not found
Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9720233779757780143
 xla_global_id: -1]

## Lectura de datos

In [ ]:
!cp /content/drive/MyDrive/Programas/DataReader.py /content
!cp /content/drive/MyDrive/Programas/Decoding.py /content
!cp /content/drive/MyDrive/Programas/DistributedTraining.py /content
!cp /content/drive/MyDrive/Programas/EncodingClass.py /content
!cp /content/drive/MyDrive/Programas/Operators.py /content

In [ ]:
# Ruta del dataset
!cp /content/drive/MyDrive/Dataset/Images.zip /content

In [ ]:
# Descomprimir dataset
!unzip Images.zip

In [ ]:
## MÓDULOS
from Operators import *
from EncodingClass import *
from Decoding import *
from DataReader import *
from DistributedTraining import *

In [ ]:
## MÓDULOS ##########################################################################################
from Operators import *
from EncodingClass import *
from Decoding import *
from DataReader import *
from DistributedTraining import *

## LIBRERÍAS ########################################################################################
import numpy as np
from torch import optim
import pandas as pd
import timeit
import torch
from torch import nn
from multiprocessing import Process, Manager
import pickle

## HIPERPARÁMETROS DE LA CNN ########################################################################
loss_func = nn.NLLLoss(reduction = "sum") #Defining loss function
lr = 1e-4 #Defining learning rate
#Maximun and minimum numbers of layers to initialize networks
min_conv = 2
max_conv = 5
min_full = 1
max_full = 4
w = 0.3 #penalization weight
max_params = 2e6
num_epochs = 10

## PARÁMETROS GA ####################################################################################
N = 20 #Population size
mr = 0.5 #Mutation rate
T = 50 #Number of generations
t_size = 5 #tournament size
train_dl, test_dl = loading_data()
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## CREAR POBLACIÓN ##################################################################################
pop = []
bestAcc = []
bestF = []
bestParams = []
print("Iniciando población...\n")
while len(pop) < N:
  #while True:
    #try:
      acc_list = []
      #Creating genomes (genetic encoding)
      e1 = Encoding(min_conv,max_conv,min_full,max_full)
      #Decoding the network
      network1 = decoding(e1)
      #Creating the CNN
      cnn1 = CNN(e1, network1[0], network1[1], network1[2])
      print("Individuo {} creado:".format(len(pop)+1))
      #Training the CNN
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc_list)
      #Add individual to population
      pop.append([e1,acc_list[0][1],acc_list[0][2],acc_list[0][3]])
      print("Individuo {} evaluado.\n".format(len(pop)))
print("Población creada!!")

In [ ]:
#Mostrar aptitudes, precisión y num. parámetros de población inicial
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

In [ ]:
# Control deterministico
def funcion_dinámica(Pc,Gen,GenMax):
  return Pc=(Gen/GenMax)**(1/2)
cr = 0

T = 50 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)

  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')

    parents.append(p1)
    parents.append(p2)

    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    ##### Función dinámica------------
    cr = funcion_dinámica(cr,t,T)      #  <------------------------------------Función dinámica------------
    ##### Función dinámica------------
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)

    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)

    if mr >= random.uniform(0,1):
      mutation(c2)
    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]

  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])


  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

In [ ]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)

  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')

    parents.append(p1)
    parents.append(p2)

    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)

    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)

    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]

  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])


  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

In [ ]:
#Mostrar aptitudes, precisión y num. parámetros de población final
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

In [ ]:
results = pd.DataFrame(list(zip(bestAcc, bestF, bestParams)), columns = ['Accuracy', 'Fitness', 'No. Params'])
final_networks = []
final_connections = []
objects = []
for member in pop:
    p = member[0]
    objects.append(p)
    n_conv = p.n_conv
    n_full = p.n_full
    description = 'The network has ' + str(n_conv) + ' convolutional layers ' + 'with: '
    for i in range(n_conv):
        nfilters = str(p.first_level[i]['nfilters'])
        fsize = str(p.first_level[i]['fsize'])
        pool = str(p.first_level[i]['pool'])
        psize = str(p.first_level[i]['psize'])
        layer = '(' + nfilters + ', ' + fsize + ', ' + pool + ', ' + psize + ') '
        description += layer
    description += 'and '
    description += str(n_full)
    description += ' '
    description += 'fully-connected layers with: '
    for i in range(n_conv, n_conv+n_full):
        neurons = str(p.first_level[i]['neurons'])
        layer = '(' + neurons + ')'
        description += layer
    description += ' neurons'
    final_networks.append(description)

    connections = ''
    for bit in p.second_level:
        if bit == 1:
            connections += 'one - '
        if bit == 0:
            connections += 'zero - '
    final_connections.append(connections)


final_population = pd.DataFrame(list(zip(final_networks, final_connections)), columns = ['Network Architecture', 'Connections'])